In [2]:
import numpy as np
import re

# get list of experiments

In [18]:
gst_experiment_list = open("rb_experiment_list_dir/data/dataset.txt", "r")
exp_list = list(gst_experiment_list)
gst_experiment_list.close()

In [21]:
gatelists = []
for expline in exp_list[1:]:
    gateline = expline.split('@')[0]
    gatelist = gateline.split(':Q0')[:-1]
    gatelists.append(gatelist)
# gatelists[0] = ["empty"]

In [22]:
gatelists

[['Gzpi2', 'Gxpi2', 'Gzpi2', 'Gzpi2', 'Gxpi2', 'Gzpi2'],
 ['Gzpi2', 'Gzpi2', 'Gzpi2', 'Gzpi2'],
 ['Gzpi2',
  'Gxpi2',
  'Gzpi2',
  'Gzpi2',
  'Gzpi2',
  'Gzpi2',
  'Gxpi2',
  'Gzpi2',
  'Gxpi2',
  'Gxpi2'],
 ['Gxpi2', 'Gxpi2', 'Gxpi2', 'Gxpi2'],
 ['Gzpi2', 'Gxpi2', 'Gxpi2', 'Gxpi2', 'Gxpi2', 'Gzpi2', 'Gzpi2', 'Gzpi2'],
 ['Gzpi2',
  'Gzpi2',
  'Gzpi2',
  'Gxpi2',
  'Gxpi2',
  'Gzpi2',
  'Gzpi2',
  'Gzpi2',
  'Gxpi2',
  'Gxpi2'],
 ['Gxpi2',
  'Gzpi2',
  'Gzpi2',
  'Gxpi2',
  'Gxpi2',
  'Gxpi2',
  'Gzpi2',
  'Gzpi2',
  'Gxpi2',
  'Gxpi2'],
 ['Gxpi2', 'Gzpi2', 'Gzpi2', 'Gzpi2', 'Gzpi2', 'Gxpi2', 'Gxpi2', 'Gxpi2'],
 ['Gxpi2',
  'Gzpi2',
  'Gzpi2',
  'Gzpi2',
  'Gzpi2',
  'Gxpi2',
  'Gzpi2',
  'Gzpi2',
  'Gzpi2',
  'Gzpi2',
  'Gxpi2',
  'Gxpi2'],
 ['Gzpi2',
  'Gxpi2',
  'Gzpi2',
  'Gzpi2',
  'Gzpi2',
  'Gxpi2',
  'Gxpi2',
  'Gzpi2',
  'Gxpi2',
  'Gxpi2',
  'Gxpi2',
  'Gzpi2'],
 ['Gxpi2',
  'Gzpi2',
  'Gxpi2',
  'Gxpi2',
  'Gxpi2',
  'Gzpi2',
  'Gxpi2',
  'Gzpi2',
  'Gzpi2',
  'Gzpi2'],
 ['Gz

In [23]:
len(gatelists)

66

# generate master scripts

$Gx$ should include a z rotation to remove phase acculumated since last $Gx$ due to detuning of drive. $Gx_0(\pi/2)$ defined to be a pi/2 around x:

$$
Gx(\pi/2) = Gx_0(\pi/2)~ exp(-i \theta\sigma_z/2)
$$

For qubit splitting from field $\Delta_0$, and time $t$ since last $Gx$, $\theta = 2\pi \Delta_0 t$. And for splitting $\Delta_z$ under light shift for Z, the time of the Z part of the composite X is 

$$
t_z = \frac{\Delta_0}{\Delta_z} t
$$

$$
Gx(\pi/2) = Gx_0(\pi/2)~ Gz(2\pi \Delta_z t_z)
$$

In [43]:
gatelistnames = []
for j, gatelist in enumerate(gatelists):
    gatelist_new = []
    for gate in gatelist:
        gatelist_new.append(gate.split('pi2')[0].split('G')[1])
    gatelistnames.append("".join(gatelist_new))

In [49]:
f = open("qubit_empty_script.mScript", "r")
flist = list(f)

zpi2time = 0.00049
z2pitime = 0.00196
xpi2time = 0.00022
delta0 = 1.25671917
detlaz = 1/z2pitime

Gz = ["on: zrot\n","t += zpi2time\n","off: zrot\n", "t += 0.001\n"]

for j, gatelist in enumerate(gatelists):
    td = 0
    tz = 0
    f0 = open("rb_scripts/"+gatelistnames[j]+".mScript", "w")
    flist_new = ["%"+gatelistnames[j]+"\n"]
    for i, line in enumerate(flist):
        if (line == "%STARTQUBIT\n"):
            flist_new += flist[:i+1].copy() 
            for gate in gatelist:
                if (gate == 'Gzpi2'):
                    td += zpi2time
                    flist_new += Gz
                if (gate == "Gxpi2"):
                    tz = np.around(z2pitime - td*(delta0/detlaz),5)
                    Gx = ["on: zrot\n","t += "+str(tz)+"\n","off: zrot\n","t += 0.001\n","on: xrot\n","t += xpi2time\n","off: xrot\n","t += 0.001\n"]
                    flist_new += Gx
                    td = xpi2time + tz
                td += 0.001 
            flist_new += flist[i+1:]
            break
    for line in flist_new:
        f0.write(line)
    f0.close()
f.close()

fexplist = open("rb_scripts/experiment_gate_list.txt", "w")
for j, gatelist in enumerate(gatelists):
    fexplist.write("".join(gatelist)+"\n")
fexplist.close()